In [1]:
# pip install openai
# !pip install --upgrade openai

In [2]:
from openai import OpenAI
from keys import openai_key

# Function to use OpenAI autocomplete
def generate_autocomplete(question, temperature=0, top_p=1, n=1, system_prompt="You are a helpful assistant."):
    """
    Generates an autocomplete response using OpenAI's GPT model.
    
    Parameters:
        prompt_text (str): The initial text to complete.
        max_tokens (int): The Max number of tokens to generate in the completion.
        temperature (float): Controls the creativity (0.0 = deterministic, higher = more creative).
        top_p (float): Controls the probability threshold for vocabulary sampling.
        n (int): Number of completions to generate.
    
    Returns:
        str: The generated completion text.
    """
    # Make sure to set your OpenAI API key here
    client = OpenAI(
        # This is the default and can be omitted
        api_key=openai_key,
        )
    
    try:
        # Use the ChatCompletion API (chat-based interface)
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": question}
            ],
        )
        
        # Extract and return the generated text from the response
        return response.choices[0].message.content.strip()
    
    except Exception as e:
        return f"Error: {e}"


# Example usage
# question = "What is the capital of France?"
# result = generate_autocomplete(question)
# print(result)


In [3]:
# task = """You are an intelligent SQL assistant. Your job is to generate SQL queries based on the provided database schema.

# Rules:
# 1. Always respond with the SQL query inside a JSON object using the format: `{"query": "---query---"}`.
# 2. Ensure the query is based strictly on the tables and data provided in the schema.
# 3. If a query requests information not available in the tables, return `{"query": ""}` (an empty query).
# 4. The SQL query must be syntactically correct and optimized for performance.
# 5. Avoid making assumptions about the data beyond what is provided in the schema.
# 6. Do not include any comments or explanations in the response—just the JSON object with the query.

# You are a precise SQL assistant that only returns SQL queries formatted in JSON.

# """

task = """You are an intelligent SQL assistant. Your job is to generate SQL queries based on the provided database schema.
# Rules:
# 1. Ensure the query is based strictly on the tables and data provided in the schema.
# 2. The SQL query must be syntactically correct and optimized for performance.
# 3. Avoid making assumptions about the data beyond what is provided in the schema.
# 4. Get data in 2024 to optimize the query run time if there is no specific time.
# 5. Limit 30 rows.

COT:
Step 1: Understand the Question
    Goal: Break down the user’s natural language question into components: What information is requested? What tables/fields might be involved?
    Action:
    Parse the question to identify the key entities, conditions, and expected output.
    Classify the question: Is it a simple retrieval (SELECT)? Does it involve filtering (WHERE)? Does it require joining multiple tables (JOIN)?
Step 2: Map the Question to Database Schema
    Goal: Relate the parsed components to the actual database tables, columns, and relationships.
    Action:
    Look up in the provided schema which tables and columns hold the data relevant to the question.
    Determine how the tables are connected (e.g., via foreign keys).
    Example:
    If the question is “What is the banner information for Sponsor X?”, identify which table holds banners and how sponsors are related.
Step 3: Plan SQL Query Structure
    Goal: Decide on the SQL structure based on the parsed question and mapped schema.
    Action:
    Formulate the SQL query type (SELECT, JOIN, WHERE, etc.).
    If the question involves multiple conditions (filters), ensure to include them using appropriate WHERE clauses.
    If tables need to be joined, determine the JOIN types (INNER JOIN, LEFT JOIN, etc.).
Step 4: Generate SQL Query
    Goal: Construct the SQL query based on the mappings and conditions.
    Action:
    Write the SQL query step-by-step.
    Ensure the correct fields and tables are selected, proper joins are made, and any conditions (filters) are applied.

"""

# read schema from file tables.txt
with open("tables.txt", "r") as f:
    schema = f.read()

# read examples from file examples.txt
with open("examples.txt", "r") as f:
    examples = f.read()

# Combine the task description, schema, and examples
system_prompt = task + schema + examples

In [11]:
question = "count companies signed up to the platform Sep in 2024. date_joined is not in table users_company"
result = generate_autocomplete(question, system_prompt=system_prompt)
print(result)

To count the number of companies signed up to the platform in September 2024, we need to look at the `users_company` table and join it with the `users_account` table, where the `date_joined` field is stored. The `date_joined` field in the `auth_user` table represents when user accounts were created, which can be linked to when a company signed up.

Here’s the SQL query to achieve that:

```sql
SELECT 
    COUNT(DISTINCT company.id) AS total_companies_signed_up  -- Count distinct company IDs
FROM 
    users_company AS company                           -- Main table containing company data
INNER JOIN 
    users_account AS account ON company.id = account.company_id  -- Join to link company with user accounts
INNER JOIN 
    auth_user AS user ON account.user_id = user.id  -- Join to link user account with auth_user table
WHERE 
    user.date_joined >= '2024-09-01'                 -- Filter for users who joined on or after September 1, 2024
    AND user.date_joined < '2024-10-01';          